<a href="https://colab.research.google.com/github/sahiser19/ST-554-Project1-Sarah_Ryan_Heather/blob/Task-3---Heather/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

## Aims and Goals

The main goal of this portion of the data analysis is to determine which model of regression is best for our data set.

To do this we will predict the benzene concentration given a variety of variables. We are going to try and then test two different regression models.

The first model is a single linear regression model which uses the concentration of carbon dioxide in the air to predict the concentration of benzene in the air.

The second model is a multipule linear regression model which looks at carbon dioxide concentration, temperature, relative humidity, and absolute humidity to predict the concentration of benzene in the air.

One complicating factor of this data is that it is run over a period of time. Since it is gathered sequentially it makes sense to analyze the data sequentially. This is due to the fact that two days in a row may have very similar values. To do this we will have to write our own cross validation function.

This report will contain an initial fit of the data. The cross validation function written along with results and interpritations. Finally, a final fit will be done.

## Read in Data

All coding starts with reading in and cleaning up the data. This data was gathered using

In [7]:
!pip install ucimlrepo
import ucimlrepo as uci
air_quality = uci.fetch_ucirepo(id=360)
air_quality.data.features

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


## Clean Data

#Initial Fit

#Cross Validation

##Helper Function

##Cross Validation Function

# Final Fit